<a href="https://colab.research.google.com/github/lol782/RAG-powered-chatbot-for-Kumaoni-language-preservation/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This is just for testing**

In [ ]:
import json
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter

# Load dataset
with open("kumaoni_euttaranchal_dataset.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# Create smaller chunks with metadata
docs = []
for i, item in enumerate(raw_data):
    if isinstance(item.get("english"), str) and isinstance(item.get("kumaoni"), str):
        text = f"English: {item['english']} | Kumaoni: {item['kumaoni']}"
        doc = Document(page_content=text, metadata={"source": f"entry_{i}"})
        docs.append(doc)

# Optional: Split longer chunks (if sentences are long)
text_splitter = CharacterTextSplitter(separator="|", chunk_size=256, chunk_overlap=20)
split_docs = text_splitter.split_documents(docs)

# Embed using Gemini embeddings
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(split_docs, embedding)
vectorstore.save_local("kumaoni_faiss_index")


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Load vectorstore & retriever
retriever = FAISS.load_local("kumaoni_faiss_index", embedding, allow_dangerous_deserialization=True).as_retriever()
retriever.search_kwargs["k"] = 5  # Fetch top 5 relevant chunks

# Prompt Template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a helpful and culturally aware Kumaoni assistant. Your goal is to provide friendly, conversational, and complete answers in the **Kumaoni language**.

Use the context below to guide your tone and vocabulary, but feel free to respond with natural Kumaoni sentences — not just direct translations. You may include greetings or polite phrases to make the conversation feel warm and local.

When appropriate, explain things like a local guide or elder would, giving real-life examples from Kumaoni culture, traditions, or places.

Always reply only in Kumaoni.

Context:
{context}

User: {question}
Kumaoni Response (Only answer in Kumaoni, no explanation):
"""
)

# Gemini LLM
llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash", temperature=0.4)

# RetrievalQA with Sources (for debugging/testing)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    response = qa_chain.run(user_input)
    print("Kumaoni Bot:", response)


In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio python-dotenv


In [ ]:
from pyngrok import ngrok
import nest_asyncio

In [ ]:
import uvicorn
nest_asyncio.apply()

# Start ngrok tunnel for port 8000
ngrok.set_auth_token("2uXZquOIfZsWQskGbFfztHHOjjy_4czZ4azSRn6AUv9kLZew2")  # Replace with your ngrok token
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Start FastAPI server
# The first argument to uvicorn.run should be the FastAPI app object, not the file name.
uvicorn.run(app, host="0.0.0.0", port=8000) # Changed app.py to app

## Create a fastapi application

### Subtask:
Define a FastAPI app instance.


**Reasoning**:
Import the FastAPI class and create an instance of the FastAPI application.



In [ ]:
from fastapi import FastAPI

app = FastAPI()

## Define a pydantic model for input

### Subtask:
Create a Pydantic model to validate the incoming request data, which will be the user's query.


**Reasoning**:
Define a Pydantic model to validate the incoming request data based on the instructions.



In [ ]:
from pydantic import BaseModel

class Query(BaseModel):
    user_query: str

## Create an api endpoint

### Subtask:
Define an asynchronous POST endpoint that accepts the user query using the defined Pydantic model.


**Reasoning**:
Define an asynchronous POST endpoint that accepts the user query using the defined Pydantic model.



In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

class Query(BaseModel):
    user_query: str

@app.post("/kumaoni_query")
async def kumaoni_query_endpoint(query: Query):
    pass


## Integrate langchain components

### Subtask:
Inside the API endpoint, load the necessary Langchain components (retriever, LLM, QA chain) and use them to process the user query.


**Reasoning**:
Implement the logic within the API endpoint to load Langchain components and process the user query.



**Reasoning**:
The previous command failed because `langchain_community` was not found. Although `langchain-community` was installed in a previous cell, the current runtime might not have it available. Re-installing it within this cell will ensure it's available for the code to run.



In [ ]:
!pip install -U langchain-community

**Reasoning**:
Re-execute the code to load the necessary Langchain components and define the API endpoint, now that `langchain-community` is confirmed to be installed.



**Reasoning**:
The previous command failed because `langchain_google_genai` was not found. Re-installing it within this cell will ensure it's available for the code to run.



In [ ]:
!pip install -U langchain_google_genai

**Reasoning**:
Re-execute the code to load the necessary Langchain components and define the API endpoint, now that `langchain_google_genai` is confirmed to be installed.



**Reasoning**:
The previous command failed because the `faiss` package was not found, despite attempts to install `faiss-cpu` and `langchain-community`. Re-installing `faiss-cpu` explicitly within this cell will ensure it's available for the code to run.



In [ ]:
!pip install -U faiss-cpu

**Reasoning**:
Re-execute the code to load the necessary Langchain components and define the API endpoint, now that `faiss-cpu` is confirmed to be installed.



In [ ]:
import json, os
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware   # 🔹 add
from pydantic import BaseModel
from langchain.vectorstores import FAISS
from langchain_google_genai import (GoogleGenerativeAIEmbeddings,
                                    ChatGoogleGenerativeAI)
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from google.colab import userdata
from pyngrok import ngrok
import nest_asyncio, uvicorn

# ── API key ──────────────────────────────────────────────────────────
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# ── FastAPI app ──────────────────────────────────────────────────────
app = FastAPI()

# 🔹  CORS middleware  ────────────────────────────────────────────────
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],            # ← in prod, replace * with your site
    allow_methods=["*"],            # allow POST / OPTIONS
    allow_headers=["*"],
)

# ── Pydantic model ───────────────────────────────────────────────────
class Query(BaseModel):
    user_query: str

# ── Build / load vector index (runs once) ────────────────────────────
with open("kumaoni_euttaranchal_dataset.json", encoding="utf-8") as f:
    raw = json.load(f)

docs = [
    Document(page_content=f"English: {item['english']} | Kumaoni: {item['kumaoni']}")
    for item in raw
    if isinstance(item.get("english"), str) and isinstance(item.get("kumaoni"), str)
]

emb = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
index_path = "kumaoni_faiss_index"

if os.path.exists(index_path):
    vectorstore = FAISS.load_local(index_path, emb, allow_dangerous_deserialization=True)
else:
    vectorstore = FAISS.from_documents(docs, emb)
    vectorstore.save_local(index_path)

retriever = vectorstore.as_retriever()

# ── Prompt & chain ───────────────────────────────────────────────────
template = """
You are a helpful and culturally aware Kumaoni assistant. Your goal is to provide friendly, conversational, and complete answers in the **Kumaoni language**.

Use the context below to guide your tone and vocabulary, but feel free to respond with natural Kumaoni sentences — not just direct translations. You may include greetings or polite phrases to make the conversation feel warm and local.

When appropriate, explain things like a local guide or elder would, giving real-life examples from Kumaoni culture, traditions, or places.

Always reply only in Kumaoni.
Context:
{context}

User: {question}
Kumaoni Response:
"""
prompt = PromptTemplate(input_variables=["context", "question"], template=template)

llm   = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash", temperature=0.4)
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
)

# ── Endpoint ─────────────────────────────────────────────────────────
@app.post("/kumaoni_query")
async def kumaoni_query(q: Query):
    return {"answer": chain.run(q.user_query)}

**Reasoning**:
Modify the return statement in the `kumaoni_query_endpoint` function to return a dictionary containing the Kumaoni response, which FastAPI will serialize into JSON.



## Update the uvicorn run command

### Subtask:
Modify the uvicorn run command to use the newly created FastAPI app instance.


**Reasoning**:
Modify the uvicorn run command to use the newly created FastAPI app instance.



**Reasoning**:
The previous command failed because the `pyngrok` module was not found. I need to install `pyngrok` to fix this.



In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn

nest_asyncio.apply()

# Start ngrok tunnel for port 8000
ngrok.set_auth_token("********")  # Replace with your ngrok token
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Start FastAPI server
uvicorn.run(app, host="0.0.0.0", port=8000)